In [72]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
import re

In [34]:
data = pd.read_pickle('data/all_tainan_data.pkl')
data.reset_index(drop=True, inplace=True) 

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173978 entries, 0 to 173977
Data columns (total 25 columns):
base_area                173978 non-null float64
building_area            173978 non-null float64
category                 173978 non-null object
construction_type        173978 non-null object
date                     173978 non-null object
height                   173978 non-null float64
latitude                 173978 non-null float64
license_category         173978 non-null object
longitude                173978 non-null float64
number_basements         173978 non-null int64
number_design_changes    173978 non-null int64
number_floors            173978 non-null int64
number_households        173978 non-null int64
total_floor_area         173978 non-null float64
damage_level             173978 non-null int64
liquefaction             173978 non-null int64
e_PGA                    173978 non-null float64
e_PGV                    173978 non-null float64
e_PGD                  

In [97]:
x_train = data[['base_area', 'building_area', 'category', 'construction_type', 'date', 'height', 'latitude'\
               ,'license_category', 'longitude', 'number_basements', 'number_design_changes', 'number_floors'\
              ,'number_households', 'total_floor_area', 'liquefaction','e_PGA', 'e_PGV', 'e_PGD', 'n_PGA', 'n_PGV'\
              ,'n_PGD', 'u_PGA', 'u_PGV', 'u_PGD']]
y_train = data['damage_level']

In [87]:
x_train.loc[120557]

base_area                    113.13
building_area                 66.96
category                        old
construction_type               RC造
date                     000年00月03日
height                        10.65
latitude                    23.1344
license_category                use
longitude                   120.294
number_basements                  0
number_design_changes             0
number_floors                     3
number_households                 1
total_floor_area             218.74
liquefaction                      1
e_PGA                       12.9178
e_PGV                       12.1147
e_PGD                       11.3287
n_PGA                       15.3959
n_PGV                       14.2452
n_PGD                       15.1051
u_PGA                       1.90597
u_PGV                       2.46272
u_PGD                       3.77236
Name: 120557, dtype: object

In [98]:
#deal with dates
dates = []
for item in x_train.date:
    split_date = item.split('年')
    numeric_year = split_date[0]
    split_date = split_date[1].split('月')
    numeric_month = split_date[0]
    split_date = split_date[1].split('日')
    numeric_day = split_date[0]
    if numeric_year == '' or numeric_year == '***':
        numeric_year = '8'
    if numeric_month == '' or numeric_day == '*' or numeric_month == '**':
        numeric_month = '0'
    if numeric_day == '' or numeric_day == '*' or numeric_day == '**':
        numeric_day = '0'
    if numeric_year == '109' or numeric_year == '108':#
        numeric_year == '107'
    numeric_year = int(numeric_year) * 365
    numeric_month = int(numeric_month) * 30
    numeric_day = int(numeric_day)
    if numeric_year == 0:
        numeric_year = 8 * 365
    numeric_date = numeric_year + numeric_month + numeric_day
    dates += [numeric_date]
    
x_train['date'] = dates

/home/xavier/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [92]:
x_train.date[-10:]

173968     2920
173969    39200
173970    39381
173971     2920
173972    39373
173973     2920
173974     2920
173975    39268
173976     2920
173977     2920
Name: date, dtype: int64

In [99]:
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

In [100]:
#one hot encoding for categorical labels
print(x_train.category.value_counts())
category_encoder = LabelEncoder()
category_encoder.fit(x_train.category)
x_train['category'] = label_encode(category_encoder, x_train['category'])
x_train.category.value_counts()

old    173978
Name: category, dtype: int64


/home/xavier/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1.0    173978
Name: category, dtype: int64

In [ ]:
#todo represent this differently
print(x_train.construction_type.value_counts())
construction_type_encoder = LabelEncoder()
construction_type_encoder.fit(x_train.construction_type)
x_train['construction_type'] = label_encode(construction_type_encoder, x_train['construction_type'])
x_train.construction_type.value_counts()

In [ ]:
print(x_train.license_category.value_counts())
license_category_encoder = LabelEncoder()
license_category_encoder.fit(x_train.license_category)
x_train['license_category'] = label_encode(license_category_encoder, x_train['license_category'])
x_train.license_category.value_counts()

In [ ]:
y_train_encoder = LabelEncoder()
y_train_encoder.fit(y_train)
y_train = label_encode(construction_type_encoder, y_train)
y_train[:10]

In [ ]:
#prepare x_train
x_train = x_train.values

In [ ]:
# I/O check
input_shape = x_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(y_train_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=128)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=128)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [ ]:
epochs = 5
batch_size = 32

# training!
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
print('training finish')